In [1]:
%load_ext autoreload
%autoreload 2

import os

import tqdm
from rich import print
from rich.traceback import install

import datasets

os.environ["HYDRA_FULL_ERROR"] = "1"
install()

<bound method InteractiveShell.excepthook of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x7f7a11f98550>>

In [2]:
# ! pip install -e ../
# ! mamba install -c conda-forge itables pandas -y
# ! pip install pytorchvideo
import copy
import json
import os
import pathlib
import random
import sys
import time
from collections import defaultdict
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from dataclasses import dataclass
from enum import Enum
from math import floor
from typing import Any, Callable, Dict, List, Optional, Tuple, Union
import datasets

import numpy as np
import pandas as pd
import PIL
import pyarrow as pa
import pyarrow.dataset as ds
import pyarrow.parquet as pq
import torch
import torch.nn as nn
import tqdm
from hydra_zen import builds, instantiate
from PIL import Image
from pytorchvideo.data.encoded_video import EncodedVideo
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    ShortSideScale,
    UniformTemporalSubsample,
)
from rich import print
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.dataloader import default_collate
from torchvision.transforms import Compose, RandomCrop, Resize, ToTensor
from torchvision.transforms._transforms_video import CenterCropVideo
from traitlets import default
from transformers import CLIPModel, CLIPProcessor
import datasets
from tali_wit.data import (
    AnyModalSample,
    dataclass_collate,
    default_image_transforms,
    ModalityTypes,
    select_subtitles_between_timestamps,
    TALIDataset,
)
from tqdm.notebook import tqdm

/home/evolvingfungus/miniconda/envs/minimal-ml-template/lib/python3.10/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/home/evolvingfungus/miniconda/envs/minimal-ml-template/lib/python3.10/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [3]:
from tali_wit.data_plus import TALIBase
from tali_wit.data import (
    AnyModalSample,
    dataclass_collate,
    default_image_transforms,
    ModalityTypes,
    get_base_modality,
    select_subtitles_between_timestamps,
)
from rich import print

num_samples = 4

dataset_show = TALIBase(
    set_name="train",
    tali_dataset_dir="/data_fast/tali-v-3-4/",
    modality_list=[
        ModalityTypes.wit_image.value,
        ModalityTypes.wit_caption.value,
        ModalityTypes.wit_title.value,
        ModalityTypes.wit_main_body.value,
        ModalityTypes.youtube_title.value,
        ModalityTypes.youtube_image.value,
        ModalityTypes.youtube_video.value,
        ModalityTypes.youtube_subtitles.value,
        ModalityTypes.youtube_audio.value,
        ModalityTypes.youtube_description.value,
    ],
    rng_seed=42,
    top_k_tali=10,
    image_size=224,
    num_video_frames=10,
    num_audio_frames=16000 * 2,
    clip_duration_in_seconds=3,
    deterministic_sampling=True,
    dummy_batch_mode=False,
    image_text_model_name="openai/clip-vit-base-patch16",
    audio_model_name="openai/whisper-base",
    use_model_preprocessing=False,
    num_samples_per_episode=num_samples,
    total_num_samples=1000000,
    cache_generated_samples_in_memory=True,
    cache_num_samples=10,
)

dataset_predict = TALIBase(
    set_name="train",
    tali_dataset_dir="/data_fast/tali-v-3-4/",
    modality_list=[
        ModalityTypes.wit_image.value,
        ModalityTypes.wit_caption.value,
        ModalityTypes.wit_title.value,
        ModalityTypes.wit_main_body.value,
        ModalityTypes.youtube_title.value,
        ModalityTypes.youtube_image.value,
        ModalityTypes.youtube_video.value,
        ModalityTypes.youtube_subtitles.value,
        ModalityTypes.youtube_audio.value,
        ModalityTypes.youtube_description.value,
    ],
    rng_seed=42,
    top_k_tali=10,
    image_size=224,
    num_video_frames=10,
    num_audio_frames=16000 * 2,
    clip_duration_in_seconds=3,
    deterministic_sampling=True,
    dummy_batch_mode=False,
    image_text_model_name="openai/clip-vit-base-patch16",
    audio_model_name="openai/whisper-base",
    use_model_preprocessing=True,
    num_samples_per_episode=num_samples,
    total_num_samples=1000000,
    cache_generated_samples_in_memory=True,
    cache_num_samples=10,
)

# sample = dataset[0]

# for k, v in sample.items():
#     print(k, type(v))

In [4]:
from tali_wit.decorators import configurable
from tali_wit.utils import get_logger, load_json, save_json
from tali_wit.models import ModalityConfig
import torchvision
import torchaudio
import gradio as gr


cache_dir = "/data_fast/.demo/cache"


logger = get_logger(__name__)


def update_modality_panel(modality_type):
    if modality_type == "text":
        gr.update(children=[gr.inputs.Textbox(label="Text")])
    elif modality_type == "image":
        gr.update(children=[gr.inputs.Image(label="Image")])
    elif modality_type == "video":
        gr.update(children=[gr.inputs.Video(label="Video")])
    elif modality_type == "audio":
        gr.update(children=[gr.inputs.Audio(label="Audio")])
    else:
        raise ValueError(f"Unknown modality type: {modality_type}")


def video_array_to_video_file(i, video_array):
    video_path = f"{cache_dir}/{i}.mp4"

    if not pathlib.Path(video_path).parent.exists():
        pathlib.Path(video_path).parent.mkdir(parents=True, exist_ok=True)

    torchvision.io.write_video(video_path, video_array, fps=3)
    return video_path


def audio_array_to_audio_file(i, audio_array):
    audio_path = f"{cache_dir}/{i}.mp3"

    if not pathlib.Path(audio_path).parent.exists():
        pathlib.Path(audio_path).parent.mkdir(parents=True, exist_ok=True)

    torchaudio.save(audio_path, audio_array.view(-1).unsqueeze(0), 16000)

    return audio_path


def fetch_random_samples(button):
    mapper_dict = {
        "wikipedia_caption_text": "wit-caption",
        "wikipedia_caption_image": "wit-images",
        "youtube_content_video": "tali-videos",
        "youtube_content_audio": "tali-audio",
        "youtube_random_video_sample_image": "tali-images",
        "youtube_description_text": "tali-description",
        "youtube_title_text": "tali-title",
        "youtube_subtitle_text": "tali-subtitles",
    }
    reverse_mapper_dict = {v: k for k, v in mapper_dict.items()}
    idx = np.random.randint(0, len(dataset))
    sample = dataset[idx]
    images = {"wit-images": [], "tali-images": []}
    videos = {"tali-videos": []}
    audio = {"tali-audio": []}
    text = {
        "wit-caption": [],
        "tali-title": [],
        "tali-description": [],
        "tali-subtitles": [],
    }

    for key in videos:
        for i in range(num_samples):
            video_array = sample[reverse_mapper_dict[key]][i].permute(
                0, 2, 3, 1
            )
            video_path = video_array_to_video_file(i, video_array)

            videos[key].append(gr.update(value=video_path))

    for key in audio:
        for i in range(num_samples):
            audio_array = sample[reverse_mapper_dict[key]][i]
            audio_path = audio_array_to_audio_file(i, audio_array)
            audio[key].append(gr.update(value=audio_path))

    for key in images:
        for i in range(num_samples):
            # print(f"{key} {i} {sample[reverse_mapper_dict[key]][i]}")
            images[key].append(
                gr.update(
                    value=np.array(
                        sample[reverse_mapper_dict[key]][i].permute(1, 2, 0)
                    )
                )
            )

    for key in text:
        for i in range(num_samples):
            text[key].append(
                gr.update(value=sample[reverse_mapper_dict[key]][i])
            )

    image_list = [[entry for entry in item] for item in images.values()]
    image_list = [item for sublist in image_list for item in sublist]
    video_list = [[entry for entry in item] for item in videos.values()]
    video_list = [item for sublist in video_list for item in sublist]
    audio_list = [[entry for entry in item] for item in audio.values()]
    audio_list = [item for sublist in audio_list for item in sublist]
    text_list = [[entry for entry in item] for item in text.values()]
    text_list = [item for sublist in text_list for item in sublist]

    return *image_list, *video_list, *audio_list, *text_list


def rank_by_random(text):
    mapper_dict = {
        "wikipedia_caption_text": "wit-caption",
        "wikipedia_caption_image": "wit-images",
        "youtube_content_video": "tali-videos",
        "youtube_content_audio": "tali-audio",
        "youtube_random_video_sample_image": "tali-images",
        "youtube_description_text": "tali-description",
        "youtube_title_text": "tali-title",
        "youtube_subtitle_text": "tali-subtitles",
    }
    reverse_mapper_dict = {v: k for k, v in mapper_dict.items()}
    sample = dataset.mem_cache[-1]

    images = {"wit-images": [], "tali-images": []}
    videos = {"tali-videos": []}
    audio = {"tali-audio": []}
    text = {
        "wit-caption": [],
        "tali-title": [],
        "tali-description": [],
        "tali-subtitles": [],
    }
    shuffle_idx = np.random.permutation(num_samples)
    for key in videos:
        for i in shuffle_idx:
            video_array = sample[reverse_mapper_dict[key]][i].permute(
                0, 2, 3, 1
            )
            video_path = video_array_to_video_file(i, video_array)

            videos[key].append(gr.update(value=video_path))

    for key in audio:
        for i in shuffle_idx:
            audio_array = sample[reverse_mapper_dict[key]][i]
            audio_path = audio_array_to_audio_file(i, audio_array)
            audio[key].append(gr.update(value=audio_path))

    for key in images:
        for i in shuffle_idx:
            # print(f"{key} {i} {sample[reverse_mapper_dict[key]][i]}")
            images[key].append(
                gr.update(
                    value=np.array(
                        sample[reverse_mapper_dict[key]][i].permute(1, 2, 0)
                    )
                )
            )

    for key in text:
        for i in shuffle_idx:
            text[key].append(
                gr.update(value=sample[reverse_mapper_dict[key]][i])
            )

    image_list = [[entry for entry in item] for item in images.values()]
    image_list = [item for sublist in image_list for item in sublist]
    video_list = [[entry for entry in item] for item in videos.values()]
    video_list = [item for sublist in video_list for item in sublist]
    audio_list = [[entry for entry in item] for item in audio.values()]
    audio_list = [item for sublist in audio_list for item in sublist]
    text_list = [[entry for entry in item] for item in text.values()]
    text_list = [item for sublist in text_list for item in sublist]

    return *image_list, *video_list, *audio_list, *text_list

In [5]:
from tali_wit.utils import download_model_with_name
from accelerate import Accelerator
import yaml

accelerator = Accelerator(mixed_precision="bf16")
path_dict = download_model_with_name(hf_cache_dir="demo_weights/", 
                                     model_name="ckpt_22015", 
                                     hf_repo_path="Antreas/tali-2-tali_omni_base_patch16_224-wit_tali_image_text_audio_video_dataset-2306")

with open(path_dict["config_filepath"], "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
model = instantiate(config["model"])
model = accelerator.prepare(model)

print(f"Loading model from {path_dict['root_filepath']}")

accelerator.load_state(path_dict["root_filepath"])

Loading model from demo_weights/checkpoints/ckpt_22015

In [6]:
# from tali_wit.data_plus import generate_hierarchical_data_dict

# fetch_samples = dataset_predict[0]
# sample = dataset_predict.mem_cache[-1]
# hierarchical_sample = generate_hierarchical_data_dict(sample)
# hierarchical_sample = accelerator.prepare(hierarchical_sample)
# for key, value in hierarchical_sample.items():
#     for subkey, subvalue in value.items():
#         hierarchical_sample[key][subkey] = accelerator.prepare(subvalue)
#         # print(hierarchical_sample[key][subkey])

# output_dict = model.forward(hierarchical_sample)

# def get_similarities_with_source(source_name):
#     filter_dict = {}
#     for key in output_dict:
#         if "similarities" in key and source_name in key:
#             if source_name in key.split("_to_")[1]:
#                 new_key = f"{key.split('_to_')[1]}_to_{key.split('_to_')[0]}"
#                 filter_dict[new_key] = output_dict[key].permute(1, 0)
#             else:
#                 filter_dict[key] = output_dict[key]
#     return filter_dict
                

# test_output = get_similarities_with_source("wikipedia_caption_text")

# for key in test_output:
#     print(key, test_output[key].shape)

In [7]:
# def rank_by_text(text):
#     mapper_dict = {
#         "wikipedia_caption_text": "wit-caption",
#         "wikipedia_caption_image": "wit-images",
#         "youtube_content_video": "tali-videos",
#         "youtube_content_audio": "tali-audio",
#         "youtube_random_video_sample_image": "tali-images",
#         "youtube_description_text": "tali-description",
#         "youtube_title_text": "tali-title",
#         "youtube_subtitle_text": "tali-subtitles",
#     }
#     reverse_mapper_dict = {v: k for k, v in mapper_dict.items()}
#     sample = dataset.mem_cache[-1]

#     images = {"wit-images": [], "tali-images": []}
#     videos = {"tali-videos": []}
#     audio = {"tali-audio": []}
#     text = {
#         "wit-caption": [],
#         "tali-title": [],
#         "tali-description": [],
#         "tali-subtitles": [],
#     }
    
#     output_dict = model.forward(sample)
    
#     shuffle_idx = np.random.permutation(num_samples)
#     for key in videos:
#         for i in shuffle_idx:
#             video_array = sample[reverse_mapper_dict[key]][i].permute(
#                 0, 2, 3, 1
#             )
#             video_path = video_array_to_video_file(i, video_array)

#             videos[key].append(gr.update(value=video_path))

#     for key in audio:
#         for i in shuffle_idx:
#             audio_array = sample[reverse_mapper_dict[key]][i]
#             audio_path = audio_array_to_audio_file(i, audio_array)
#             audio[key].append(gr.update(value=audio_path))

#     for key in images:
#         for i in shuffle_idx:
#             # print(f"{key} {i} {sample[reverse_mapper_dict[key]][i]}")
#             images[key].append(
#                 gr.update(
#                     value=np.array(
#                         sample[reverse_mapper_dict[key]][i].permute(1, 2, 0)
#                     )
#                 )
#             )

#     for key in text:
#         for i in shuffle_idx:
#             text[key].append(
#                 gr.update(value=sample[reverse_mapper_dict[key]][i])
#             )

#     image_list = [[entry for entry in item] for item in images.values()]
#     image_list = [item for sublist in image_list for item in sublist]
#     video_list = [[entry for entry in item] for item in videos.values()]
#     video_list = [item for sublist in video_list for item in sublist]
#     audio_list = [[entry for entry in item] for item in audio.values()]
#     audio_list = [item for sublist in audio_list for item in sublist]
#     text_list = [[entry for entry in item] for item in text.values()]
#     text_list = [item for sublist in text_list for item in sublist]

#     return *image_list, *video_list, *audio_list, *text_list

In [8]:
callback = gr.CSVLogger()

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    source = gr.Markdown("### Input Source", interactive=False)
    description_source_modality = gr.Markdown(
        "Select the type of input modality you want to use to rank the results from the tabs",
        interactive=False,
    )
    description_source_type = gr.Markdown(
        "Once you do, next, choose the way you wish to input the data. You can either upload a file, use your webcam, or draw on the canvas.",
        interactive=False,
    )
    description_rank = gr.Markdown(
        "Once you have selected a source modality, then selected an input source, and finally uploaded your data, click the Rank button",
        interactive=False,
    )
    with gr.Row():
        with gr.Column():
            with gr.Tab("text"):
                with gr.Tab("generic"):
                    generic_text = gr.Textbox(label="Text")
                    rank_by_generic_text_button = gr.Button(
                        "Rank Results", label="Rank Results"
                    )
                with gr.Tab("caption"):
                    caption_text = gr.Textbox(label="Caption")
                    rank_by_caption_text_button = gr.Button(
                        "Rank Results", label="Rank Results"
                    )
                with gr.Tab("title"):
                    title_text = gr.Textbox(label="Title")
                    rank_by_title_text_button = gr.Button(
                        "Rank Results", label="Rank Results"
                    )
                with gr.Tab("description"):
                    description_text = gr.Textbox(label="Description")
                    rank_by_description_text_button = gr.Button(
                        "Rank Results", label="Rank Results"
                    )
                with gr.Tab("subtitles"):
                    subtitles_text = gr.Textbox(label="Subtitles")
                    rank_by_subtitles_text_button = gr.Button(
                        "Rank Results", label="Rank Results"
                    )

            with gr.Tab("image"):
                with gr.Tab("upload-image"):
                    uploaded_image = gr.Image(label="Image")
                    rank_by_uploaded_image_button = gr.Button(
                        "Rank Results", label="Rank Results"
                    )
                with gr.Tab("camera"):
                    recorded_image = gr.Image(source="webcam", streaming=True)
                    rank_by_recorded_image_button = gr.Button(
                        "Rank Results", label="Rank Results"
                    )
                with gr.Tab("draw"):
                    drawn_image = gr.Image(source="canvas")
                    rank_by_drawn_image_button = gr.Button(
                        "Rank Results", label="Rank Results"
                    )

            with gr.Tab("video"):
                with gr.Tab("upload-video"):
                    upload_video = gr.Video(label="Video")
                    rank_by_upload_video_button = gr.Button(
                        "Rank Results", label="Rank Results"
                    )
                with gr.Tab("camera"):
                    camera_video = gr.Video(source="webcam")
                    rank_by_camera_video_button = gr.Button(
                        "Rank Results", label="Rank Results"
                    )

            with gr.Tab("audio"):
                with gr.Tab("upload-audio"):
                    upload_audio = gr.Audio(label="Audio")
                    rank_by_audio_upload_button = gr.Button(
                        "Rank Results", label="Rank Results"
                    )
                with gr.Tab("microphone"):
                    mic_audio = gr.Audio(source="microphone")
                    rank_by_audio_mic_button = gr.Button(
                        "Rank Results", label="Rank Results"
                    )

    images = {"wit-images": [], "tali-images": []}
    videos = {"tali-videos": []}
    audio = {"tali-audio": []}
    text = {
        "wit-caption": [],
        "tali-title": [],
        "tali-description": [],
        "tali-subtitles": [],
    }

    source_content = gr.Markdown(
        "Fetch some samples to rank, by clicking the button below! 🖼️ 🎦 🔊 📝",
        interactive=False,
    )

    fetch_random_samples_button = gr.Button(
        "Fetch Random TALI Samples", label="Fetch Random Samples"
    )

    with gr.Row():
        with gr.Column():
            for key in images:
                with gr.Tab(key):
                    for i in range(num_samples):
                        images[key].append(gr.Image())

        with gr.Column():
            for key in videos:
                with gr.Tab(key):
                    for i in range(num_samples):
                        videos[key].append(gr.Video())

        with gr.Column():
            for key in audio:
                with gr.Tab(key):
                    for i in range(num_samples):
                        audio[key].append(gr.Audio())

        with gr.Column():
            for key in text:
                with gr.Tab(key):
                    for i in range(num_samples):
                        text[key].append(gr.Textbox(lines=3))

        # unpack the lists of lists into a single list, collapse the list of lists into a single list
        image_list = [[entry for entry in item] for item in images.values()]
        image_list = [item for sublist in image_list for item in sublist]
        video_list = [[entry for entry in item] for item in videos.values()]
        video_list = [item for sublist in video_list for item in sublist]
        audio_list = [[entry for entry in item] for item in audio.values()]
        audio_list = [item for sublist in audio_list for item in sublist]
        text_list = [[entry for entry in item] for item in text.values()]
        text_list = [item for sublist in text_list for item in sublist]

        fetch_random_samples_button.click(
            fetch_random_samples,
            [fetch_random_samples_button],
            [*image_list, *video_list, *audio_list, *text_list],
        )

        # rank_by_audio_mic_button.click(
        #     rank_by_audio,
        #     [rank_by_audio_mic_button],
        #     [*image_list, *video_list, *audio_list, *text_list],
        # )
        # rank_by_audio_upload_button.click(
        #     rank_by_audio,
        #     [rank_by_audio_upload_button],
        #     [*image_list, *video_list, *audio_list, *text_list],
        # )

        # rank_by_uploaded_image_button.click(
        #     rank_by_image,
        #     [rank_by_uploaded_image_button],
        #     [*image_list, *video_list, *audio_list, *text_list],
        # )
        # rank_by_drawn_image_button.click(
        #     rank_by_image,
        #     [rank_by_drawn_image_button],
        #     [*image_list, *video_list, *audio_list, *text_list],
        # )
        # rank_by_recorded_image_button.click(
        #     rank_by_image,
        #     [rank_by_recorded_image_button],
        #     [*image_list, *video_list, *audio_list, *text_list],
        # )

        # rank_by_upload_video_button.click(
        #     rank_by_video,
        #     [rank_by_upload_video_button],
        #     [*image_list, *video_list, *audio_list, *text_list],
        # )
        # rank_by_camera_video_button.click(
        #     rank_by_video,
        #     [rank_by_camera_video_button],
        #     [*image_list, *video_list, *audio_list, *text_list],
        # )

        rank_by_generic_text_button.click(
            rank_by_random,
            [rank_by_generic_text_button],
            [*image_list, *video_list, *audio_list, *text_list],
        )
        rank_by_caption_text_button.click(
            rank_by_random,
            [rank_by_caption_text_button],
            [*image_list, *video_list, *audio_list, *text_list],
        )
        rank_by_title_text_button.click(
            rank_by_random,
            [rank_by_title_text_button],
            [*image_list, *video_list, *audio_list, *text_list],
        )
        rank_by_description_text_button.click(
            rank_by_random,
            [rank_by_description_text_button],
            [*image_list, *video_list, *audio_list, *text_list],
        )
        rank_by_subtitles_text_button.click(
            rank_by_random,
            [rank_by_subtitles_text_button],
            [*image_list, *video_list, *audio_list, *text_list],
        )

In [9]:
demo.queue(concurrency_count=8)
demo.launch(share=True, debug=True, enable_queue=True)

Running on local URL:  http://127.0.0.1:7860
